In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
  temperature = 0,
  groq_api_key = "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",
  model = "llama-3.1-70b-versatile"
)

response = llm.invoke("The first president of independent India was ...")
print(response.content)

Dr. Rajendra Prasad was the first president of independent India. He served as the president from January 26, 1950, to May 13, 1962.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.shine.com/job-search/translating-software-requirements-jobs-9")

page_data = loader.load().pop().page_content
print(page_data)

96,297 Software Requirements Job Openings - Software Requirements Job Vacancies - Shine.comWelcome!Sign inRegisterBlogFraud AlertAbout UsContact UsFAQsDownload AppReport a Job PostingEmployersRecruiter India|Post JobsPartner SiteHindustantimes.comLivemint.comLivehindustan.comOTTplay.comCommercial Radio - FAB PlayContact us080-10062222contactus@shine.comFollow us© 2024 HT Media LimitedPrivacy PolicyCookie PolicyBusiness NewsEnglish NewsDisclaimerTerms & ConditionsDownload AppDownload Shine AppGet the best experience of Shine at your fingertips.For EmployersEmployer LoginConduct HackathonsHelp080-100622229am - 6pm, Mon to SatLoginRegisterNotificationNo NotificationWe will notify you once we have something for youHOMEJOBSSearchSearch by Categories City Jobs in DelhiJobs in MumbaiJobs in BangaloreJobs in ChennaiJobs in PuneView AllSkillsSales & Marketing JobsAccounting JobsEngineering JobsFinance JobsNetworking JobsView AllDesignationSoftware Engineer JobsJava Programmer JobsHr JobsProduct

In [3]:
from langchain_core.prompts import PromptTemplate

promt_extract = PromptTemplate.from_template(
  """
  ### SCRAPED TEXT FROM WEBSITE :
  {page_data}
  ### INSTRUCTION :
  The scraped text is from the carrer's page of website.
  Your job is to extract the job postings and return them in JSON format containing 
  following keys : 'role', 'experience', 'skills' and 'description'.
  Only retur the valid JSON.
  ### VALID JSON (NO PREAMBLE):
  """
)

chain_extract = promt_extract | llm
res = chain_extract.invoke(input={'page_data' : page_data})
print(res.content)

```json
[
  {
    "role": "Application Architect",
    "experience": "10 to 14 Yrs",
    "skills": [
      "Architectural Design",
      "Analysis",
      "Solution Management",
      "Project Development",
      "Software Solutions",
      "Innovation",
      "Team Leadership",
      "Team Development",
      "Stakeholder Engagement",
      "Enterprise Architecture",
      "TOGAF",
      "Zachman",
      "Programming Languages",
      "Leadership",
      "Communication",
      "Interpersonal Skills",
      "Application Architectures",
      "ProblemSolving"
    ],
    "description": "Develop and maintain the enterprise architecture strategy and framework to support business and IT alignment. Analyze business requirements to design scalable and adaptable application architectures."
  },
  {
    "role": "Staff Customer Support Analyst",
    "experience": "3 to 7 Yrs",
    "skills": [
      "Debugging",
      "Customerfacing experience",
      "Supporting software products",
      "AR Se

In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

[{'role': 'Application Architect', 'experience': '10 to 14 Yrs', 'skills': ['Architectural Design', 'Analysis', 'Solution Management', 'Project Development', 'Software Solutions', 'Innovation', 'Team Leadership', 'Team Development', 'Stakeholder Engagement', 'Enterprise Architecture', 'TOGAF', 'Zachman', 'Programming Languages', 'Leadership', 'Communication', 'Interpersonal Skills', 'Application Architectures', 'ProblemSolving'], 'description': 'Develop and maintain the enterprise architecture strategy and framework to support business and IT alignment. Analyze business requirements to design scalable and adaptable application architectures.'}, {'role': 'Staff Customer Support Analyst', 'experience': '3 to 7 Yrs', 'skills': ['Debugging', 'Customerfacing experience', 'Supporting software products', 'AR Server', 'Helix ITSM', 'Support processes', 'Verbal', 'written communication skills'], 'description': 'Provide customer support for software products and resolve customer issues in a time

In [5]:
import pandas as pd

df = pd.read_csv("app/resource/my_portfolio_data.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [6]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
  for _, row in df.iterrows():
    collection.add(documents=row["Techstack"],
                 metadatas={"links" : row["Links"]},
                 ids=[str(uuid.uuid4())]  
                   )


In [7]:
links = collection.query(query_texts=["Experience in Python", "Experience in React"], n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [8]:
type(json_res)

list

In [9]:
job = json_res[0]
job['skills']

['Architectural Design',
 'Analysis',
 'Solution Management',
 'Project Development',
 'Software Solutions',
 'Innovation',
 'Team Leadership',
 'Team Development',
 'Stakeholder Engagement',
 'Enterprise Architecture',
 'TOGAF',
 'Zachman',
 'Programming Languages',
 'Leadership',
 'Communication',
 'Interpersonal Skills',
 'Application Architectures',
 'ProblemSolving']

In [10]:
promt_email = PromptTemplate.from_template(
  """
  ### JOB DESCRIPTION:
  {job_description}
      
  ### INSTRUCTION:
  You are Arya Trivedi, a TNP student coordinator at Indian Institute of Information Technology, Surat. 
  IIIT Surat is an Institute of National Importance (INI) and a Government Funded Technical Institute (GFTI). 
  Your role involves facilitating placements and internships for students by connecting with companies. 
  Your task is to write a cold email to the client regarding the job mentioned above, 
  describing the capability of IIIT Surat students in fulfilling their needs.  
  Additionally, highlight the institute's achievements and the relevant programs that make students suitable for the role.  
  Include the most relevant details from the following links to showcase the institution's and student's strengths: {link_list}.  
  Remember, you are Arya Trivedi, TNP student coordinator at IIIT Surat.
  Don not privide a preamble.
  ### Email (NO PREAMBLE):
  """
)

chain_email = promt_email | llm
res = chain_email.invoke({"job_description" : str(job), "link_list" : links})
print(res.content)


Subject: Introducing IIIT Surat Students for Application Architect Role

Dear Hiring Manager,

I am Arya Trivedi, TNP student coordinator at Indian Institute of Information Technology, Surat, an Institute of National Importance (INI) and a Government Funded Technical Institute (GFTI). I came across the job description for an Application Architect at your esteemed organization and was impressed by the role's requirements. I believe our students at IIIT Surat possess the necessary skills and expertise to excel in this position.

Our institute offers a range of programs, including B.Tech, M.Tech, and Ph.D. in Computer Science and Engineering, as well as Information Technology, which equip our students with a strong foundation in software development, architectural design, and innovation. Our curriculum is designed to foster skills in programming languages, analysis, solution management, and team leadership, making our students well-suited for the Application Architect role.

Our students 